<a href="https://colab.research.google.com/github/HossamElsrah/Book-Buddy-AI/blob/main/telebot_With_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -qU transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Model Loading & Quantization

In [ ]:
import transformers
import torch
from torch import cuda, bfloat16

# Configure quantization settings for loading the model with less GPU memory usage
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# Load the model for causal language modeling
llm = transformers.AutoModelForCausalLM.from_pretrained(
    'microsoft/Phi-4-mini-instruct',
    trust_remote_code=False,
    quantization_config=bnb_config,
    device_map='auto'
)

tokenizer = transformers.AutoTokenizer.from_pretrained('microsoft/Phi-4-mini-instruct',trust_remote_code=True)


# Set the model in evaluation mode for inference
llm.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(200064, 3072, padding_idx=199999)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=5120, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, ou

In [ ]:
# Define your input prompt
prompt = "Translate it to Arabic 'Hello there!'"

# Tokenize the input prompt
inputs = tokenizer(prompt, return_tensors="pt").to(llm.device)

# Generate text using the model with proper settings
outputs = llm.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

# Decode and print the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the final output
print(generated_text)

Translate it to Arabic 'Hello there!' in Arabic.
"Hello there!" in Arabic would be "مرحبا بك" (marhabanak) or simply "مرحبا" (marhaban) for a more general greeting.


# Document Setup


>  Before starting the RAG pipeline stages, we need to perform document preprocessing. This typically involves:




```
1. Chunking the documents into smaller pieces
2. Creating embeddings for the chunks
```




In [ ]:
#!pip install sentence-transformers faiss-cpu PyPDF2 torch pypdf telebot
#!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00


## Load a BOOK & Chunk It

In [ ]:
import gdown
from typing import List

# --- Function for downloading the book ---
def download_book(file_id: str, output_path: str) -> str:
    # Print a message indicating the start of the download
    print(f"Start downloading book with ID '{file_id}'...")
    try:
        # Use the gdown library to download the file
        gdown.download(id=file_id, output=output_path, quiet=False)
        print(f"Book downloading done successfully: '{output_path}'")
        # Return the local path of the downloaded file
        return output_path
    except Exception as e:
        # Handle any error that may occur during download
        print(f"Error downloading file: {e}")
        return None

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from typing import List

# --- Function for processing the book and splitting into chunks ---
def process_book_for_rag(file_path: str, chunk_size: int = 1000, chunk_overlap: int = 200) -> List[Document]:
    # Print a message indicating the start of the processing and splitting
    print("\nProcessing book and splitting into chunks...")
    try:
        # Load the PDF document using PyPDFLoader
        loader = PyPDFLoader(file_path)
        data = loader.load()
        # Set up the text splitter
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,        # The size of each chunk
            chunk_overlap=chunk_overlap,    # The overlap between chunks
            separators=["\n\n", "\n", " ", ""] # The delimiters used for splitting
        )
        # Split the document into chunks
        docs = text_splitter.split_documents(data)
        print(f"The book has been split into {len(docs)} chunks.")
        # Return the list of chunks
        return docs
    except Exception as e:
        print(f"Error processing the book: {e}")
        return []

In [ ]:
# --- Phase 1: Download and Process the Book ---
print("--- Starting the RAG Pipeline Setup (Phase 1: Download and Process) ---")
file_id = '1-zfWR1dP_EViBQ5NestEKsDFgXBQ7cgP'
book_file_name = 'book.pdf'

# Step 1: Download the book
downloaded_path = download_book(file_id, book_file_name)

if downloaded_path:
    # Step 2: Process the book and get chunks
    book_chunks = process_book_for_rag(downloaded_path)
    print("\n✅ Phase 1: Download and Processing complete.")
else:
    print("\n❌ Phase 1: Failed to download the book.")

print("--- Phase 1 Finished ---")

--- Starting the RAG Pipeline Setup (Phase 1: Download and Process) ---
Start downloading book with ID '1-zfWR1dP_EViBQ5NestEKsDFgXBQ7cgP'...


Downloading...
From: https://drive.google.com/uc?id=1-zfWR1dP_EViBQ5NestEKsDFgXBQ7cgP
To: /content/book.pdf
100%|██████████| 33.1M/33.1M [00:00<00:00, 78.5MB/s]


Book downloading done successfully: 'book.pdf'

Processing book and splitting into chunks...
The book has been split into 1341 chunks.

✅ Phase 1: Download and Processing complete.
--- Phase 1 Finished ---


## Embeddings

In [ ]:
import numpy as np
import faiss
import torch
from sentence_transformers import SentenceTransformer
from langchain_core.documents import Document
from typing import List
import re

# --- Function for creating the Vector Database ---
def create_faiss_vector_store(docs: List[Document], embedding_model_name: str = "sentence-transformers/all-MiniLM-L6-v2", index_path: str = "faiss_index") -> faiss.Index:
    # Print a message indicating the start of creating embeddings and the index
    print("\nCreating embeddings and building FAISS index...")
    try:
        # Load the SentenceTransformer model to create embeddings
        model = SentenceTransformer(embedding_model_name)
        # Extract and clean the text content from the Document objects
        texts = []
        for doc in docs:
            if isinstance(doc.page_content, str):
                # Basic text cleaning: remove non-alphanumeric characters except spaces and punctuation,
                # and replace multiple spaces with a single space.
                cleaned_text = re.sub(r'[^\w\s.,!?;:]', '', doc.page_content)
                cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
                texts.append(cleaned_text)


        if not texts:
            print("Error: No valid text content found in the documents for embedding.")
            return None

        # Create embeddings for the text chunks
        print(f"Generating embeddings for {len(texts)} chunks...")
        embeddings = model.encode(texts, show_progress_bar=True, convert_to_numpy=True)

        # Build the FAISS index
        print("Building the Faiss index...")
        index = faiss.IndexFlatL2(embeddings.shape[1])
        index.add(embeddings)

        # Save the index to a local file
        faiss.write_index(index, index_path)
        print("The Vector Store has been created and saved successfully.")
        print(f"Number of vectors in the database: {index.ntotal}")

        return index
    except Exception as e:
        print(f"❌ Error creating the vector store: {e}")
        return None

In [ ]:
# --- Phase 2: Create the Vector Database ---
print("\n--- Starting the RAG Pipeline Setup (Phase 2) ---")
if 'book_chunks' in locals() and book_chunks:
    # Step 3: Create the FAISS vector store
    vector_index = create_faiss_vector_store(book_chunks)

    if vector_index:
        print("\n✅ Phase 2: Vector store creation complete.")
    else:
        print("\n❌ Phase 2: Failed to create the vector store.")
else:
    print("\n❌ Phase 2: Book chunks not available. Please run Phase 1 first.")

print("--- RAG Pipeline Setup Finished ---")


--- Starting the RAG Pipeline Setup (Phase 2) ---

Creating embeddings and building FAISS index...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generating embeddings for 1341 chunks...


Batches:   0%|          | 0/42 [00:00<?, ?it/s]

Building the Faiss index...
The Vector Store has been created and saved successfully.
Number of vectors in the database: 1341

✅ Phase 2: Vector store creation complete.
--- RAG Pipeline Setup Finished ---


# Rag Pipline


>  Here we should define several functions to test the complete pipeline and its functions:





```
1. Retrieval function
2. Re-ranking function
3. Generation function
```




## Retrieval Stage

In [ ]:
import faiss
from sentence_transformers import SentenceTransformer
from langchain_core.documents import Document
from typing import List
import re

# --- Phase 3: Retrieval ---
# --- Function for Retrieval ---
def retrieve_documents(query: str, index: faiss.Index, docs: List[Document], k: int = 3) -> List[Document]:
    """
    Performs a similarity search on the FAISS index to retrieve relevant documents.

    Args:
        query: The user's query string.
        index: The FAISS index.
        docs: The list of original Document objects.
        k: The number of nearest neighbors to retrieve.

    Returns:
        A list of the top k most relevant Document objects.
    """
    print(f"Retrieving top {k} documents for query: '{query}'")

    # Define the embedding model for retrieval

    embedding_model_for_retrieval = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    print("Embedding model for retrieval defined.")

    try:
        # Create embedding for the query
        query_embedding = embedding_model_for_retrieval.encode([query], convert_to_numpy=True)

        # Perform similarity search
        distances, indices = index.search(query_embedding, k)

        # Retrieve the corresponding documents
        retrieved_docs = [docs[i] for i in indices[0]]

        print(f"Successfully retrieved {len(retrieved_docs)} documents.")
        return retrieved_docs
    except Exception as e:
        print(f"❌ Error during document retrieval: {e}")
        return []

## Reranking Stage

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from typing import List
from langchain_core.documents import Document

# --- Phase 4: Re-ranking ---
# --- Function for Re-ranking Documents ---
def rerank_documents(query: str, documents: List[Document], reranking_model_name: str = "cross-encoder/ms-marco-MiniLM-L-6-v2") -> List[Document]:
    """
    Re-ranks a list of documents based on their relevance to the query using a cross-encoder model.

    Args:
        query: The user's query string.
        documents: A list of Document objects retrieved from the initial search.
        reranking_model_name: The name of the cross-encoder model to use for re-ranking.

    Returns:
        A list of Document objects re-ranked by relevance.
    """
    print("\nStarting document re-ranking...")
    if not documents:
        print("No documents to re-rank.")
        return []

    try:
        # Load the re-ranking model and tokenizer
        tokenizer = AutoTokenizer.from_pretrained(reranking_model_name)
        model = AutoModelForSequenceClassification.from_pretrained(reranking_model_name)
        # Ensure the model is on the correct device
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)

        # Create pairs of query and document text for the cross-encoder
        sentence_pairs = [[query, doc.page_content] for doc in documents]

        # Tokenize and get scores from the re-ranking model
        with torch.no_grad():
            inputs = tokenizer(sentence_pairs, padding=True, truncation=True, return_tensors="pt").to(device)
            scores = model(**inputs).logits.squeeze()

        # Pair documents with their scores and sort by score in descending order
        document_scores = list(zip(documents, scores.tolist()))
        document_scores_sorted = sorted(document_scores, key=lambda x: x[1], reverse=True)

        # Return the re-ranked documents
        reranked_docs = [doc for doc, score in document_scores_sorted]
        print(f"Successfully re-ranked {len(reranked_docs)} documents.")
        return reranked_docs

    except Exception as e:
        print(f"❌ Error during document re-ranking: {e}")
        return documents # Return original documents in case of error

## Generation Stage

In [ ]:
# --- Phase 5: Generation ---
# Define a function for Generation
def generate_response(query: str, documents: List[Document], llm, tokenizer) -> str:
    """
    Generates a response to the user query based on the provided documents using the language model.

    Args:
        query: The user's query string.
        documents: A list of relevant Document objects.
        llm: The loaded language model.
        tokenizer: The tokenizer for the language model.

    Returns:
        The generated response string.
    """
    # print("Generating response using LLM...") # Removed print statement
    if not documents:
        return "I could not find enough relevant information to answer your query."

    # Construct the prompt for the LLM with more specific instructions
    # This is an improved example of prompt engineering
    context = "\n\n".join([doc.page_content for doc in documents])
    prompt = f"""You are an AI assistant that answers questions based on the provided context from a book.
                 Use ONLY the information from the following context to answer the user's question.
                 If the answer cannot be found in the context, please respond with "I cannot answer this question based on the provided information."

    Context:
    {context}

    Question: {query}

    Answer:"""

    try:
        # Tokenize the input prompt
        inputs = tokenizer(prompt, return_tensors="pt").to(llm.device)

        # Generate text using the model
        outputs = llm.generate(
            **inputs,
            max_new_tokens=500, # Increased the number of max_new_tokens
            do_sample=True,
            top_k=50,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

        # Decode and return the generated text
        # We skip the prompt tokens in the output
        generated_text = tokenizer.decode(outputs[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True)

        # --- Post-processing: Remove trailing numbers and associated punctuation ---
        # This regex looks for a space followed by optional punctuation (.,!?;:) then one or more digits at the end of the string
        # Or just one or more digits at the very end
        cleaned_text = re.sub(r'[\s.,!?;:]*\d+$', '', generated_text).strip()
        # --- End of Post-processing ---

        # print("Response generated successfully.") # Removed print statement
        return cleaned_text

    except Exception as e:
        print(f"❌ Error during response generation: {e}")
        return "Sorry, I could not generate a response at this time."

# Pipline Testing

In [ ]:
# --- End-to-End RAG Pipeline Execution (Retrieval -> Re-ranking -> Generation) ---
print("\n--- Starting End-to-End RAG Pipeline Execution ---")

# Example User Query
user_query = "What is the Linear Regression?"
k_retrieval = 10 # Number of documents to retrieve initially
k_reranked = 3  # Number of documents to keep after re-ranking for generation

# Ensure prerequisites are met (vector_index, book_chunks, embedding_model_for_retrieval, llm, tokenizer)
if 'vector_index' in locals() and vector_index is not None and 'book_chunks' in locals() and book_chunks \
   and 'embedding_model_for_retrieval' in locals() and embedding_model_for_retrieval is not None \
   and 'llm' in locals() and llm is not None and 'tokenizer' in locals() and tokenizer is not None:

    print(f"\nProcessing query: '{user_query}'")

    # Step 1: Retrieve initial documents
    initial_retrieved_documents = retrieve_documents(user_query, vector_index, book_chunks, k=k_retrieval)

    if initial_retrieved_documents:
        print(f"\nRetrieved {len(initial_retrieved_documents)} initial documents.")
        # Step 2: Re-rank the retrieved documents
        if 'rerank_documents' in locals():
            reranked_documents = rerank_documents(user_query, initial_retrieved_documents)

            if reranked_documents:
                print(f"\nRe-ranked documents. Selecting top {k_reranked}.")
                # Step 3: Generate response using the top re-ranked documents
                final_response = generate_response(user_query, reranked_documents[:k_reranked], llm, tokenizer)

                print("\n--- Final Generated Response ---")
                print(final_response)
                print("\n✅ End-to-End RAG Pipeline Execution complete.")
            else:
                print("\n❌ Re-ranking failed.")
        else:
             print("\n❌ Re-ranking function not found. Ensure rerank_documents function cell is executed.")
    else:
        print("\n❌ Initial document retrieval failed.")
else:
    print("\n❌ Prerequisites for End-to-End RAG Pipeline not met. Ensure all previous phases (Setup, Retrieval, Re-ranking, Generation function definition) were completed successfully and variables are not None.")

print("--- End-to-End RAG Pipeline Execution Finished ---")


--- Starting End-to-End RAG Pipeline Execution ---

Processing query: 'What is the Linear Regression?'
Retrieving top 10 documents for query: 'What is the Linear Regression?'
Embedding model for retrieval defined.
Successfully retrieved 10 documents.

Retrieved 10 initial documents.

Starting document re-ranking...
Successfully re-ranked 10 documents.

Re-ranked documents. Selecting top 3.

--- Final Generated Response ---
Linear Regression is a statistical method used to model the relationship between a dependent variable and one or more independent variables by fitting a linear equation to the observed data. In the example provided, life satisfaction is modeled as a dependent variable, and GDP per capita is the independent variable. The linear equation used is life_satisfaction = θ0 + θ1 × GDP_per_capita, where θ0 is the intercept and θ1 is the coefficient for GDP per capita. Linear Regression is used in this context to predict life satisfaction based on GDP per capita.

✅ End-to-En

# Telebot

In [ ]:
import telebot
import logging

# actual Telegram bot token
bot_token = "8323908843:AAEQkuXPPkgHNsaYmiaQrm_UrpkniRkB_Uk"

# --- Telegram Bot Setup ---
# Create a TelegramBot object with your bot token
bot = telebot.TeleBot(bot_token)

# Configure logging for better debugging
logging.basicConfig(level=logging.INFO)

# This handler will respond to all messages that are not commands
@bot.message_handler(func=lambda message: True)
def handle_rag_query(message):
    # Get the chat ID for sending the response
    chat_id = message.chat.id
    user_query = message.text

    logging.info(f"Received query from chat ID {chat_id}: '{user_query}'")

    # Send a waiting message
    bot.send_message(chat_id, "جاري البحث عن إجابة لسؤالك... ⏳")

    # RAG Pipeline Parameters
    k_retrieval = 10 # Number of documents to retrieve initially
    k_reranked = 3  # Number of documents to keep after re-ranking for generation

    # Ensure prerequisites are met
    required_vars = ['vector_index', 'book_chunks', 'embedding_model_for_retrieval', 'llm', 'tokenizer']
    if not all(var in globals() and globals()[var] is not None for var in required_vars):
        missing_vars = [var for var in required_vars if var not in globals() or globals()[var] is None]
        error_msg = f"عذرًا، مكونات RAG ليست محملة بالكامل. يرجى التأكد من تشغيل جميع الخلايا السابقة. المكونات المفقودة: {', '.join(missing_vars)}"
        bot.send_message(chat_id, error_msg)
        logging.error(error_msg)
        return


    try:
        # Step 1: Retrieve initial documents using the actual function
        initial_retrieved_documents = retrieve_documents(user_query, vector_index, book_chunks, k=k_retrieval)

        if not initial_retrieved_documents:
            bot.send_message(chat_id, "عذرًا، لم أجد أي معلومات ذات صلة بسؤالك في الكتاب.")
            logging.warning(f"No relevant documents found for query: '{user_query}'")
            return

        # Step 2: Re-rank the retrieved documents using the actual function
        reranked_documents = rerank_documents(user_query, initial_retrieved_documents)

        # Step 3: Generate response using the top re-ranked documents and the actual function
        final_response = generate_response(user_query, reranked_documents[:k_reranked], llm, tokenizer)

        # Send the final RAG response back to the chat
        bot.send_message(chat_id, final_response)
        logging.info(f"Response sent for query: '{user_query}'")

    except Exception as e:
        logging.error(f"An error occurred during RAG pipeline execution for query '{user_query}': {e}")
        bot.send_message(chat_id, "عذرًا، حدث خطأ أثناء معالجة طلبك. يرجى المحاولة مرة أخرى لاحقًا.")

# Start polling for incoming messages
print("Bot is now running. Send messages to your bot on Telegram.")
bot.infinity_polling()

Bot is now running. Send messages to your bot on Telegram.
Retrieving top 10 documents for query: 'hello'
Embedding model for retrieval defined.
Successfully retrieved 10 documents.

Starting document re-ranking...
Successfully re-ranked 10 documents.
Retrieving top 10 documents for query: 'what is the support vector machines ?'
Embedding model for retrieval defined.
Successfully retrieved 10 documents.

Starting document re-ranking...
Successfully re-ranked 10 documents.
Retrieving top 10 documents for query: 'what is the random forest'
Embedding model for retrieval defined.
Successfully retrieved 10 documents.

Starting document re-ranking...
Successfully re-ranked 10 documents.
Retrieving top 10 documents for query: 'what is the loss functions'
Embedding model for retrieval defined.
Successfully retrieved 10 documents.

Starting document re-ranking...
Successfully re-ranked 10 documents.
Retrieving top 10 documents for query: 'what is the loss functions'
Embedding model for retrieva

2025-08-09 23:00:15,502 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2025-08-09 23:00:15,504 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling


# Dynamic Telebot

In [ ]:
import telebot
import logging
from telebot.types import Document as TelegramDocument # Alias to avoid conflict with langchain_core.documents.Document
import os

# actual Telegram bot token
bot_token = "8323908843:AAEQkuXPPkgHNsaYmiaQrm_UrpkniRkB_Uk"

# --- Telegram Bot Setup ---
# Create a TelegramBot object with your bot token
bot = telebot.TeleBot(bot_token)

# Configure logging for better debugging
logging.basicConfig(level=logging.INFO)

# Dictionary to store vector indices and book chunks per chat ID
# This allows different chats to process different books
chat_data = {}

# --- Helper function to download file from Telegram ---
def download_telegram_file(bot_instance, file_id, file_name):
    try:
        file_info = bot_instance.get_file(file_id)
        downloaded_file = bot_instance.download_file(file_info.file_path)
        with open(file_name, 'wb') as new_file:
            new_file.write(downloaded_file)
        logging.info(f"File downloaded successfully: {file_name}")
        return file_name
    except Exception as e:
        logging.error(f"Error downloading file from Telegram: {e}")
        return None

# This handler will respond to all messages (text or document)
@bot.message_handler(func=lambda message: True, content_types=['text', 'document'])
def handle_message(message):
    chat_id = message.chat.id
    logging.info(f"Received message from chat ID {chat_id}")

    if message.document:
        # Handle document messages
        doc: TelegramDocument = message.document
        logging.info(f"Received a document: {doc.file_name}, MIME type: {doc.mime_type}")

        # Check if the document is a PDF
        if doc.mime_type == 'application/pdf':
            bot.send_message(chat_id, f"تلقيت ملف PDF: {doc.file_name}. جاري معالجته...")

            # Define a temporary file path to save the uploaded PDF
            temp_pdf_path = f"temp_{chat_id}_{doc.file_name}"

            # Download the file from Telegram
            downloaded_path = download_telegram_file(bot, doc.file_id, temp_pdf_path)

            if downloaded_path:
                try:
                    # Process the book and get chunks
                    bot.send_message(chat_id, "جاري تقسيم الكتاب إلى أجزاء صغيرة...")
                    book_chunks = process_book_for_rag(downloaded_path)

                    if book_chunks:
                        # Create the FAISS vector store
                        bot.send_message(chat_id, "جاري إنشاء قاعدة بيانات البحث (Vector Store)...")
                        vector_index = create_faiss_vector_store(book_chunks, index_path=f"faiss_index_{chat_id}")

                        if vector_index:
                            # Store the processed data for this chat ID
                            chat_data[chat_id] = {
                                'book_chunks': book_chunks,
                                'vector_index': vector_index
                            }
                            bot.send_message(chat_id, "✅ تم تجهيز الكتاب بنجاح! يمكنك الآن طرح أسئلة.")
                            logging.info(f"Book processed and vector store created for chat ID {chat_id}")
                        else:
                            bot.send_message(chat_id, "❌ فشل في إنشاء قاعدة بيانات البحث. يرجى المحاولة مرة أخرى.")
                            logging.error(f"Failed to create vector store for chat ID {chat_id}")
                    else:
                         bot.send_message(chat_id, "❌ فشل في معالجة الكتاب. يرجى التأكد من أن الملف صالح وحاول مرة أخرى.")
                         logging.error(f"Failed to process book for chat ID {chat_id}")

                except Exception as e:
                    logging.error(f"An error occurred during document processing for chat ID {chat_id}: {e}")
                    bot.send_message(chat_id, "عذرًا، حدث خطأ أثناء معالجة ملف PDF. يرجى المحاولة مرة أخرى لاحقًا.")
                finally:
                    # Clean up the temporary downloaded file
                    if os.path.exists(temp_pdf_path):
                        os.remove(temp_pdf_path)
                        logging.info(f"Cleaned up temporary file: {temp_pdf_path}")
            else:
                bot.send_message(chat_id, "❌ فشل في تنزيل ملف PDF. يرجى المحاولة مرة أخرى.")
                logging.error(f"Failed to download PDF file for chat ID {chat_id}")

        else:
            bot.send_message(chat_id, "عذرًا، أنا أقبل ملفات PDF فقط في الوقت الحالي.")
            logging.warning(f"Received non-PDF document: {doc.file_name} (MIME type: {doc.mime_type}) from chat ID {chat_id}")

    elif message.text:
        # Handle text messages as queries (existing RAG logic)
        user_query = message.text
        logging.info(f"Received query from chat ID {chat_id}: '{user_query}'")

        # Send a waiting message
        bot.send_message(chat_id, "جاري البحث عن إجابة لسؤالك... ⏳")

        # RAG Pipeline Parameters
        k_retrieval = 10 # Number of documents to retrieve initially
        k_reranked = 3  # Number of documents to keep after re-ranking for generation

        # Ensure prerequisites are met for this chat ID
        if chat_id not in chat_data or chat_data[chat_id].get('vector_index') is None or chat_data[chat_id].get('book_chunks') is None:
            bot.send_message(chat_id, "عذرًا، لم يتم تحميل أي كتاب بعد لهذه المحادثة. يرجى إرسال ملف PDF أولاً.")
            logging.warning(f"RAG prerequisites not met for chat ID {chat_id}. No book loaded.")
            return

        vector_index = chat_data[chat_id]['vector_index']
        book_chunks = chat_data[chat_id]['book_chunks']

        # Ensure llm and tokenizer are available globally (loaded from earlier cells)
        if 'llm' not in globals() or 'tokenizer' not in globals() or llm is None or tokenizer is None:
             error_msg = "عذرًا، نموذج اللغة غير محمل. يرجى التأكد من تشغيل جميع الخلايا السابقة في بيئة الكولاب."
             bot.send_message(chat_id, error_msg)
             logging.error(error_msg)
             return
        # Ensure embedding_model_for_retrieval is available globally
        if 'embedding_model_for_retrieval' not in globals() or embedding_model_for_retrieval is None:
            error_msg = "عذرًا، نموذج التضمين غير محمل. يرجى التأكد من تشغيل الخلايا السابقة في بيئة الكولاب."
            bot.send_message(chat_id, error_msg)
            logging.error(error_msg)
            return


        try:
            # Step 1: Retrieve initial documents using the actual function
            initial_retrieved_documents = retrieve_documents(user_query, vector_index, book_chunks, k=k_retrieval)

            if not initial_retrieved_documents:
                bot.send_message(chat_id, "عذرًا، لم أجد أي معلومات ذات صلة بسؤالك في الكتاب.")
                logging.warning(f"No relevant documents found for query: '{user_query}' for chat ID {chat_id}")
                return

            # Step 2: Re-rank the retrieved documents using the actual function
            # Ensure rerank_documents function is defined
            if 'rerank_documents' not in globals():
                error_msg = "عذرًا، دالة إعادة الترتيب غير محملة. يرجى التأكد من تشغيل الخلايا السابقة."
                bot.send_message(chat_id, error_msg)
                logging.error(error_msg)
                return
            reranked_documents = rerank_documents(user_query, initial_retrieved_documents)

            # Step 3: Generate response using the top re-ranked documents and the actual function
            # Ensure generate_response function is defined
            if 'generate_response' not in globals():
                 error_msg = "عذرًا، دالة التوليد غير محملة. يرجى التأكد من تشغيل الخلايا السابقة."
                 bot.send_message(chat_id, error_msg)
                 logging.error(error_msg)
                 return
            final_response = generate_response(user_query, reranked_documents[:k_reranked], llm, tokenizer)

            # Send the final RAG response back to the chat
            bot.send_message(chat_id, final_response)
            logging.info(f"Response sent for query: '{user_query}' for chat ID {chat_id}")

        except Exception as e:
            logging.error(f"An error occurred during RAG pipeline execution for query '{user_query}' in chat ID {chat_id}: {e}")
            bot.send_message(chat_id, "عذرًا، حدث خطأ أثناء معالجة طلبك. يرجى المحاولة مرة أخرى لاحقًا.")

    else:
        # Handle other content types if necessary, though the handler is set for text and document
        logging.warning(f"Received message of unhandled type from chat ID {chat_id}: {message.content_type}")
        bot.send_message(chat_id, "عذرًا، أنا أتعامل مع الرسائل النصية وملفات PDF فقط.")


# Start polling for incoming messages
print("Bot is now running. Send messages to your bot on Telegram.")
bot.infinity_polling()

Bot is now running. Send messages to your bot on Telegram.

Processing book and splitting into chunks...
The book has been split into 53 chunks.

Creating embeddings and building FAISS index...
Generating embeddings for 53 chunks...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Building the Faiss index...
The Vector Store has been created and saved successfully.
Number of vectors in the database: 53
Retrieving top 10 documents for query: 'what is the Backtracking search ?'
Embedding model for retrieval defined.
Successfully retrieved 10 documents.

Starting document re-ranking...
Successfully re-ranked 10 documents.
Retrieving top 10 documents for query: 'what is the Constraint graph'
Embedding model for retrieval defined.
Successfully retrieved 10 documents.

Starting document re-ranking...
Successfully re-ranked 10 documents.



Processing book and splitting into chunks...
The book has been split into 44 chunks.

Creating embeddings and building FAISS index...
Generating embeddings for 44 chunks...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Building the Faiss index...
The Vector Store has been created and saved successfully.
Number of vectors in the database: 43
Retrieving top 10 documents for query: 'what is the static verification'
Embedding model for retrieval defined.
Successfully retrieved 10 documents.

Starting document re-ranking...
Successfully re-ranked 10 documents.
Retrieving top 10 documents for query: 'يعني ايه Process Synchronization'
Embedding model for retrieval defined.
Successfully retrieved 10 documents.

Starting document re-ranking...
Successfully re-ranked 10 documents.
Retrieving top 10 documents for query: 'لخص اول ٨ سلايدز'
Embedding model for retrieval defined.
Successfully retrieved 10 documents.

Starting document re-ranking...
Successfully re-ranked 10 documents.

Processing book and splitting into chunks...
The book has been split into 30 chunks.

Creating embeddings and building FAISS index...
Generating embeddings for 30 chunks...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Building the Faiss index...
The Vector Store has been created and saved successfully.
Number of vectors in the database: 30
Retrieving top 10 documents for query: 'what is Decision Table Testing'
Embedding model for retrieval defined.
Successfully retrieved 10 documents.

Starting document re-ranking...
Successfully re-ranked 10 documents.


2025-08-09 23:32:35,904 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2025-08-09 23:32:35,906 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
